In [1]:
import os, sys, numpy as np, pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
from statsmodels.stats.weightstats import _zconfint_generic, _zstat_generic

sys.path.append(os.path.abspath(".."))

# internal modules (from your src/ package)
from src.utils           import SEED, set_global_seed
from src.data_generation import lm_generate_obs_data_mcar
from src.mono_debais     import (
    lm_mono_debias_estimate_mcar_crossfit,
    lm_mono_debias_budget_constrained_obtain_alpha_mcar,
    lm_mono_debias_budget_constrained_obtain_alpha_mcar_var1
)
from src.baselines       import lm_ols_estimate

In [2]:
# ------------------- 1. Global settings -------------------
SEED = 42
set_global_seed(SEED)           # Your own helper that sets both NumPy & Python RNG

# Batch sizes
n1, n2 = 1000, 10000             # Stage-1 and Stage-2 sample sizes
reps   = 5                     # Number of Stage-2 repetitions

# ------------------- 2. Model dimensions ------------------
d_x  = 5                        # Dimension of feature vector X
d_u1 = 3                        # Dimension of latent vector U1
d_u2 = 5                        # Dimension of latent vector U2

# ------------------- 3. Model parameters ------------------
theta_star = np.arange(1, d_x + 1) * 0.2        # θ*  (shape: d_x,)

# draw β* independently of θ*  ----------------------------
beta1_star = np.arange(1, d_u1 + 1) * 1      # β1* (shape d_u1,)
beta2_star = np.arange(1, d_u2 + 1) * (-0.4)    # β2* (shape d_u2,)

# ------------------- 4. Noise parameters ------------------
sigma_eps = 3.0   # here: 3.0

# ------------------- 5. MCAR / CI hyper-parameters --------
alpha_level = 0.1                          # Target coverage level
tau, c      = 2, 5                         # Budget-constraint parameters (if used later)
alpha_init  = np.array([1/3, 1/3, 1/3])    # Initial α for Stage-1

In [3]:
# Generate observed data with MCAR missingness
X1, Y1, W1_1, W2_1, V1, R1 = lm_generate_obs_data_mcar(
    n=n1,
    d_x=d_x, d_u1=d_u1, d_u2=d_u2,
    theta_star=theta_star,
    beta1_star=beta1_star,
    beta2_star=beta2_star,
    alpha=alpha_init,
    Sigma_X=None,      # Identity covariance for X; replace if needed
    Sigma_U1=None,     # Identity covariance for U1
    Sigma_U2=None,     # Identity covariance for U2
    sigma_eps=sigma_eps,
)

alpha_opt, trace_opt, _ = lm_mono_debias_budget_constrained_obtain_alpha_mcar_var1(
    X1, Y1, W1_1, W2_1, V1, R1,
    tau=tau, c=c, method="mlp"
)

print("=== Stage‑1 results ===")
print(f"n1                     : {n1}")
print(f"initial alpha          : {np.round(alpha_init, 4)}")
print(f"optimal  alpha (α*)    : {np.round(alpha_opt,  4)}")
print(f"trace(Cov)_opt         : {trace_opt:.6f}")

# baseline α with α₂=0
alpha1_base = tau / c
alpha_baseline = np.array([alpha1_base, 0.0, 1.0 + (c - 1.0) * alpha1_base - tau])
print(f"baseline alpha (α₂=0)  : {np.round(alpha_baseline, 4)}")


c:\Users\15694\miniconda3\envs\sdenv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


=== Stage‑1 results ===
n1                     : 1000
initial alpha          : [0.3333 0.3333 0.3333]
optimal  alpha (α*)    : [0.3824 0.0881 0.5296]
trace(Cov)_opt         : 66.587473
baseline alpha (α₂=0)  : [0.4 0.  0.6]


In [4]:
# ------------------------------------------------------------
# containers
# ------------------------------------------------------------
err_opt, err_base, err_ols = [], [], []     # ℓ²-errors
len_opt, len_base          = [], []         # CI length, first coord
cov_opt, cov_base          = [], []         # coverage for first coord

for rep in range(reps):
    # =================  Batch-A : alpha_opt  ==========================
    set_global_seed(SEED + rep)
    X2A, Y2A, W1A, W2A, V2A, R2A = lm_generate_obs_data_mcar(
        n=n2,
        d_x=d_x, d_u1=d_u1, d_u2=d_u2,        # dimensions
        theta_star=theta_star,                # θ*
        beta1_star=beta1_star,                # β1*
        beta2_star=beta2_star,                # β2*
        alpha=alpha_opt,                      # MCAR mixing weights
        Sigma_X=None, Sigma_U1=None, Sigma_U2=None,  # identity covariances
        sigma_eps=sigma_eps,                  # ε noise std
    )

    theta_opt, cov_opt_mat = lm_mono_debias_estimate_mcar_crossfit(
        X2A, Y2A, W1A, W2A, V2A, R2A,
        alpha=alpha_opt, method="mlp"
    )
    theta_opt  = np.asarray(theta_opt)
    cov_opt_mat = np.asarray(cov_opt_mat)

    se_opt_1 = np.sqrt(cov_opt_mat[0, 0] / n2)            # ← first coord
    ci_opt_low, ci_opt_high = _zconfint_generic(
        theta_opt[0], se_opt_1,
        alpha=alpha_level, alternative="two-sided"
    )

    # =================  Batch-B : baseline alpha  =====================
    X2B, Y2B, W1B, W2B, V2B, R2B = lm_generate_obs_data_mcar(
        n=n2,
        d_x=d_x, d_u1=d_u1, d_u2=d_u2,
        theta_star=theta_star,
        beta1_star=beta1_star,
        beta2_star=beta2_star,
        alpha=alpha_baseline,                 # baseline MCAR weights
        Sigma_X=None, Sigma_U1=None, Sigma_U2=None,
        sigma_eps=sigma_eps,
    )
    theta_base, cov_base_mat = lm_mono_debias_estimate_mcar_crossfit(
        X2B, Y2B, W1B, W2B, V2B, R2B,
        alpha=alpha_baseline, method="mlp"
    )
    theta_base  = np.asarray(theta_base)
    cov_base_mat = np.asarray(cov_base_mat)

    se_base_1 = np.sqrt(cov_base_mat[0, 0] / n2)           # ← first coord
    ci_base_low, ci_base_high = _zconfint_generic(
        theta_base[0], se_base_1,
        alpha=alpha_level, alternative="two-sided"
    )

    # OLS baseline (same Batch-B)
    theta_ols = lm_ols_estimate(X2B, Y2B)

    # ===============  metrics for this replicate  =====================

    # 1. ℓ²-error for full θ
    err_opt .append(float(np.linalg.norm(theta_opt  - theta_star)))
    err_base.append(float(np.linalg.norm(theta_base - theta_star)))
    err_ols .append(float(np.linalg.norm(theta_ols  - theta_star)))

    # 2. CI length for θ₁
    len_opt .append(float(ci_opt_high  - ci_opt_low))
    len_base.append(float(ci_base_high - ci_base_low))

    # 3. coverage indicator for θ₁
    cov_opt .append(int(ci_opt_low  <= theta_star[0] <= ci_opt_high))
    cov_base.append(int(ci_base_low <= theta_star[0] <= ci_base_high))

# ------------------------------------------------------------
# quick numerical summary (θ₁ coverage / length)
# ------------------------------------------------------------
print(f"Mean L2-error (opt-alpha)  : {np.mean(err_opt):.4f}")
print(f"Mean L2-error (base-alpha) : {np.mean(err_base):.4f}")
print(f"Mean L2-error (OLS)        : {np.mean(err_ols):.4f}\n")

print(f"Mean CI length θ₁ (opt)  : {np.mean(len_opt):.4f}")
print(f"Mean CI length θ₁ (base) : {np.mean(len_base):.4f}\n")

print(f"Coverage θ₁ (opt)  : {np.mean(cov_opt):.3f}")
print(f"Coverage θ₁ (base) : {np.mean(cov_base):.3f}")


Mean L2-error (opt-alpha)  : 0.1492
Mean L2-error (base-alpha) : 0.1312
Mean L2-error (OLS)        : 0.1618

Mean CI length θ₁ (opt)  : 0.2532
Mean CI length θ₁ (base) : 0.2579

Coverage θ₁ (opt)  : 0.800
Coverage θ₁ (base) : 1.000


In [5]:
# ------------------------------------------------------------
# 1. Summary table  (err_* have been filled in the MC loop)
# ------------------------------------------------------------
summary = pd.DataFrame(
    {
        "mean_l2": [
            np.mean(err_opt),      # θ̂  using opt-alpha
            np.mean(err_base),     # θ̂  using base-alpha
            np.mean(err_ols),      # θ̂  OLS
        ],
        "se": [
            np.std(err_opt,  ddof=1) / np.sqrt(reps),
            np.std(err_base, ddof=1) / np.sqrt(reps),
            np.std(err_ols,  ddof=1) / np.sqrt(reps),
        ],
    },
    index=["θ̂ (opt-alpha)", "θ̂ (base-alpha)", "θ̂ (OLS)"],
)
display(summary)

# ======================================================================
# 2. Plots  — CI length  and  coverage  versus ρ
#     length_table  and  coverage_table  are shape (len(rhos), 2)
#     column 0 → opt-alpha,   column 1 → base-alpha
# ======================================================================

# ---------- CI length -------------------------------------------------
plt.figure(figsize=(8, 4))
plt.plot(rhos, length_table[:, 0], label='opt-alpha',  marker='o')
plt.plot(rhos, length_table[:, 1], label='base-alpha', marker='o')
plt.xlabel(r'Distribution shift $||\theta - \tilde\theta||$', fontsize=16)
plt.ylabel('CI length', fontsize=16)
plt.legend(prop={'size': 14})
plt.tight_layout()
plt.savefig('../results/length_rho_shift.pdf')
plt.show()

# ---------- Coverage --------------------------------------------------
plt.figure(figsize=(8, 4))
plt.plot(rhos, coverage_table[:, 0], label='opt-alpha',  marker='o')
plt.plot(rhos, coverage_table[:, 1], label='base-alpha', marker='o')
plt.ylim(0.5, 1.05)
coverage_target = 1.0 - alpha_level        # nominal level, e.g. 0.90
plt.axhline(coverage_target, color="#888888",
            linestyle='dashed', linewidth=3, zorder=1)
plt.xlabel(r'Distribution shift $||\theta - \tilde\theta||$', fontsize=16)
plt.ylabel('Coverage', fontsize=16)
plt.legend(prop={'size': 14})
plt.tight_layout()
plt.savefig('../results/coverage_rho_shift.pdf')
plt.show()


,mean_l2,se
θ̂ (opt-alpha),0.149175,0.032365
θ̂ (base-alpha),0.131171,0.037654
θ̂ (OLS),0.161785,0.048872


NameError: name 'rhos' is not defined

<Figure size 800x400 with 0 Axes>